<a href="https://colab.research.google.com/github/Valuery/Japanese-License-Plate/blob/main/Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

I. Generate dữ liệu train+ Augmentation **phụ**(Rotate nhẹ) từ dữ liệu gốc, chủ yếu tạo ra các ảnh có các góc rotate khác nhau


In [ ]:
pip install Augmentor

You should consider upgrading via the '/home/anlab/anaconda3/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import Augmentor

In [ ]:
#Đưa dữ liệu gốc vào pipeline để generate
p = Augmentor.Pipeline("/content/drive/MyDrive/Recognition CRNN/Recognition part. CRNN KERAS/dataset/Pipeline_train_and_test/train_real_dataset/") 

In [ ]:
#Thực hiện rotate mỗi ảnh 5 độ, pro = 0.7
p.rotate(probability=0.7, max_left_rotation=5, max_right_rotation=5)

In [ ]:
# Sinh ngẫu nhiên 10000 ảnh
p.sample(10000)

II. Đổi tên ảnh của dữ liệu train

In [ ]:
import os

In [ ]:
train_real_dataset_new_path = '/content/drive/MyDrive/Recognition CRNN/Recognition part. CRNN KERAS/dataset/Pipeline_train_and_test/train_output/'
train_real_dataset_new_file = os.listdir(train_real_dataset_new_path)
len(train_real_dataset_new_file)

10000

In [ ]:
count = 0
for image in train_real_dataset_new_file:
    if image.startswith('val_real_dataset_original'): 
        count = count + 1
        path = train_real_dataset_new_path + image
        print(path)
        name = image.split('.')[0].split('_')[4] + str(count) +'.png'
        print(name)
        des_path = train_real_dataset_new_path + name
        os.rename(path, des_path)

/home/anlab/License_plate/Recogniton_License_Area_Finetune_With_Real_Dataset_And_Augment/val_real_dataset/output_valid/val_real_dataset_original_名古屋.png_cf7da807-e5dc-4bda-aa50-5ca8a68d5e5c.png
名古屋1.png
/home/anlab/License_plate/Recogniton_License_Area_Finetune_With_Real_Dataset_And_Augment/val_real_dataset/output_valid/val_real_dataset_original_大阪6.png_b10073d0-fc6e-4604-8af7-471f077ff45f.png
大阪62.png
/home/anlab/License_plate/Recogniton_License_Area_Finetune_With_Real_Dataset_And_Augment/val_real_dataset/output_valid/val_real_dataset_original_なにわ.png_cf7c46cc-d768-415e-ab45-14a76b5df730.png
なにわ3.png
/home/anlab/License_plate/Recogniton_License_Area_Finetune_With_Real_Dataset_And_Augment/val_real_dataset/output_valid/val_real_dataset_original_名古屋.png_b16284ab-ac4f-423c-b589-c18cd9a5e1d0.png
名古屋4.png
/home/anlab/License_plate/Recogniton_License_Area_Finetune_With_Real_Dataset_And_Augment/val_real_dataset/output_valid/val_real_dataset_original_足立.png_7d3d2479-baec-47aa-8eb3-04bc054a233a

III. Thực hiện augmentation **chính** trên tập dữ liệu vừa được generate
- Sử dụng các phép biến đổi: Rotate, MotionBlur, ImpulseNoise, Perspective, Contrast, Brightness, Pixelate, Color

In [ ]:
from PIL import Image
import random
import numpy as np
from PIL import Image
import cv2

from straug.blur import GaussianBlur, DefocusBlur, MotionBlur, GlassBlur, ZoomBlur
from straug.camera import Contrast, Brightness, JpegCompression, Pixelate
from straug.geometry import Rotate, Perspective, Shrink, TranslateX, TranslateY
from straug.noise import GaussianNoise, ShotNoise, ImpulseNoise, SpeckleNoise
from straug.pattern import VGrid, HGrid, Grid, RectGrid, EllipseGrid
from straug.process import Posterize, Solarize, Invert, Equalize, AutoContrast, Sharpness, Color
from straug.warp import Curve, Distort, Stretch
from straug.weather import Fog, Snow, Frost, Rain, Shadow

In [ ]:
train_real_dataset_new_path = '/content/drive/MyDrive/Recognition CRNN/Recognition part. CRNN KERAS/dataset/Pipeline_train_and_test/train_output/'
train_real_dataset_new_file = os.listdir(train_real_dataset_new_path)

In [ ]:
# Thực hiện augmentation trên bộ dataset 10k ảnh đã được generate lúc đầu

for image in train_real_dataset_new_file: 
    path = train_real_dataset_new_path + image

    img = Image.open(path)
    # img = img.resize((472,236), Image.BICUBIC)
    choice = random.uniform(0,1)
    choice_mag = random.randint(0, 2)
    
    rotate = True
    angle = 0
    try: 
        if choice > 0.2: 
            img, angle = Rotate()(img, mag=choice_mag)
            # img = ZoomBlur()(img, mag=1)
            rotate = False
        choice = random.uniform(0,1)
        if choice > 0.2: 
            img = MotionBlur()(img, mag=0)
            img = ImpulseNoise()(img, mag=0)
        choice = random.uniform(0,1)
        if choice > 0.2 and rotate == True:
            img = Perspective()(img,1)
        # choice = random.uniform(0,1)
        # choice_mag = random.randint(0,2)
        # if choice > 0.6: img = Snow()(img, mag=0)
        # choice = random.uniform(0,1)
        # choice_mag = random.randint(0,2)
        # if choice > 0.9 : img = Frost(img, mag=0)
        choice = random.uniform(0,1)
        choice_mag = random.randint(0,2)
        # if choice > 0.8 : img = Rain(img, mag = 0)
        if choice > 0.5: img = Contrast()(img, mag=choice_mag)
        choice = random.uniform(0,1)
        choice_mag = random.randint(0,2)
        if choice > 0.8 : img = Brightness(img, mag=choice_mag)
        # choice = random.uniform(0,1)
        # choice_mag = random.randint(0,2)
        # if choice > 0.2 : img = JpegCompression()(img, mag=choice_mag)
        choice = random.uniform(0,1)
        choice_mag = random.randint(0,2)
        if choice > 0.5 : img = Pixelate()(img, mag=choice_mag)
        choice = random.uniform(0,1)
        choice_mag = random.randint(0,2)
        if choice > 0.4 : img = Color()(img, mag=choice_mag)
    except:
        pass
    image = image[0:len(image)-4]
    img = np.array(img)
    cv2.imwrite('/content/drive/MyDrive/Recognition CRNN/Recognition part. CRNN KERAS/dataset/Pipeline_train_and_test/train_output/{}.jpg'.format(str(image)), cv2.cvtColor(img, cv2.COLOR_RGB2BGR))

KẾT THÚC PHẦN GENERATE ẢNH TRAIN 10k ẢNH, THỰC HIỆN TƯƠNG TỰ VỚI TẬP VALIDATION, NHƯNG GENERATE 1k ẢNH, KHÔNG THỰC HIỆN AUGMENTATION **CHÍNH** TRÊN TẬP VALIDATION

IV. LOAD DATASET VÀ ĐƯA VÀO CÁC BATCH - FILE IMAGE_GENERATOR.PY 
**Trong code có chú thích augmenatation bằng tiếng việt**

In [ ]:
import cv2
import os, random
import numpy as np
from parameter import letters
from albumentations import (
    Compose, RandomBrightness, JpegCompression, HueSaturationValue, RandomContrast, HorizontalFlip,
    Rotate, Affine
)
import tensorflow as tf
# # Input data generator

# chuyển đổi label(dạng mảng) sang text
def labels_to_text(labels):     # letters의 index -> text (string)
    return ''.join(list(map(lambda x: letters[int(x)], labels)))
# chuyển đổi text sang label(dạng mảng)
def text_to_labels(text):      # text를 letters 배열에서의 인덱스 값으로 변환
    return list(map(lambda x: letters.index(x), text))


class TextImageGenerator:
    def __init__(self, img_dirpath, img_w, img_h,
                 batch_size, downsample_factor, max_text_len=3):
        self.img_h = img_h
        self.img_w = img_w
        self.batch_size = batch_size
        self.max_text_len = max_text_len
        self.downsample_factor = downsample_factor
        self.img_dirpath = img_dirpath                  # image dir path
        #self.img_dir = [f for f in os.listdir(img_dirpath) if f.endswith('.jpg') or f.endswith('.png')]     # images list
        self.img_dir = []
        for f in os.listdir(img_dirpath):
          if f.endswith('.jpg') or f.endswith('.png'):
            self.img_dir.append(f)
        self.n = len(self.img_dir)                      # number of images
        self.indexes = list(range(self.n))
        self.cur_index = 0
        self.imgs = np.zeros((self.n, self.img_h, self.img_w))
        self.texts = []
    #
    #
    # thêm hàm augmentation function sử dụng albumentation
    #
    #
    def aug_fn(self, image):
        transforms = Compose([
                            Rotate(limit=10),
                            RandomBrightness(limit=0.1),
                            # JpegCompression(quality_lower=85, quality_upper=100, p=0.5),
                            HueSaturationValue(hue_shift_limit=20, sat_shift_limit=30, val_shift_limit=20, p=0.5),
                            RandomContrast(limit=0.2, p=0.5),
                            Affine(shear=(-10,10))
        ])
        aug_img = transforms(image=image)
        aug_img = aug_img['image']
        # aug_img = tf.cast(aug_img/255.0, tf.float32)
        # aug_img = tf.image.resize(aug_img, size=[img_size, img_size])
        return aug_img

    def process_data(self, image):
        aug_img = tf.numpy_function(func=self.aug_fn, inp=image, Tout=tf.float32)
        return aug_img

    ## samples의 이미지 목록들을 opencv로 읽어 저장하기, texts에는 label 저장
    def build_data(self, d_set):
        print(self.n, " Image Loading start...")
        # print(len(self.img_dir))
        for i, img_file in enumerate(self.img_dir):
            try:
                img = cv2.imread(self.img_dirpath + img_file, cv2.IMREAD_GRAYSCALE)
                img = cv2.resize(img, (self.img_w, self.img_h))
                img = img.astype(np.float32)
                img = (img / 255.0) * 2.0 - 1.0

                self.imgs[i, :, :] = img
                if d_set == "train":
                    #print(img_file)
                    # if len(img_file[:-10])!=8 and len(img_file[:-10])!=9:
                    #     print(img_file)
                    
                    # if len(img_file[:-10]) == 8:
                    #     pad = img_file
                        
                    #     img_file = 'Z' + pad
                        
                    
                    # self.texts.append(img_file[:-10])
                    # if ' ' in img_file[:-10]:
                    #     print(img_file[:-10])
                    # print(img_file)
                    
                    string = ''

                    for i in range(3):
                      if img_file[i]>='0' and img_file[i]<='9':
                        continue
                      else:
                        string += img_file[i]
                    if string.endswith('p'): string = string.rstrip(string[-1])
                    if string.endswith('.'): string = string.rstrip(string[-1])
                  
                    # if len(image_file)!=11 and len(image_file)!=12:
                    #     print(image_file)
                    if len(string) == 2:
                        pad = string
                        string = 'Z' + pad
                    if len(string) == 1:
                        pad = string
                        string = 'Z' + 'Z' + pad
                    self.texts.append(string)
                    # if ' ' in image_file:
                    #     print(image_file)

                elif d_set == "val":
                    # print(img_file)
                    
                    string = ''
                    for i in range(3):
                      if img_file[i]>='0' and img_file[i]<='9':
                        continue
                      else:
                        string += img_file[i]
                    if string.endswith('p'): string = string.rstrip(string[-1])
                    if string.endswith('.'): string = string.rstrip(string[-1])
                    # if len(image_file)!=11 and len(image_file)!=12:
                    #     print(image_file)
                    if len(string) == 2:
                        pad = string
                        string = 'Z' + pad
                    if len(string) == 1:
                        pad = string
                        string = 'Z' + 'Z' + pad
                    self.texts.append(string)
                 
            except:
                print("fuck! " + str(img_file))
        print(len(self.texts))
        print(len(self.texts) == self.n)
        print(self.n, " Image Loading finish...")


    def next_sample(self):      ## index max -> 0 으로 만들기
        self.cur_index += 1
        if self.cur_index >= self.n:
            self.cur_index = 0
            random.shuffle(self.indexes)
        return self.imgs[self.indexes[self.cur_index]], self.texts[self.indexes[self.cur_index]]


    def next_batch(self):       ## batch size만큼 가져오기
        while True:
            X_data = np.ones([self.batch_size, self.img_w, self.img_h, 1])     # (bs, 128, 64, 1)
            Y_data = np.ones([self.batch_size, self.max_text_len])             # (bs, 9)
            input_length = np.ones((self.batch_size, 1)) * (self.img_w // self.downsample_factor - 2)  # (bs, 1)
            label_length = np.zeros((self.batch_size, 1))           # (bs, 1)

            

            for i in range(self.batch_size):
                img, text = self.next_sample()
                #
                #
                # Thực hiện augmentation mỗi ảnh trong batch 
                img = self.aug_fn(img)
                #
                #
                #
                img = img.T
                img = np.expand_dims(img, -1)
                
                X_data[i] = img
                
                #if len(text_to_labels(text)) == 9:
                #    print(text)
                if '_' in text:
                    print(text)
                Y_data[i] = text_to_labels(text)
                label_length[i] = len(text)
            # dict 형태로 복사
            inputs = {
                'the_input': X_data,  # (bs, 128, 64, 1)
                'the_labels': Y_data,  # (bs, 8)
                'input_length': input_length, 
                'label_length': label_length  
            }
            outputs = {'ctc': np.zeros([self.batch_size])}   # (bs, 1) -> 모든 원소 0
            yield (inputs, outputs)



    def next_batch_val(self):       ## batch size만큼 가져오기
        while True:
            X_data = np.ones([self.batch_size, self.img_w, self.img_h, 1])     # (bs, 128, 64, 1)
            Y_data = np.ones([self.batch_size, self.max_text_len])             # (bs, 9)
            input_length = np.ones((self.batch_size, 1)) * (self.img_w // self.downsample_factor - 2)  # (bs, 1)
            label_length = np.zeros((self.batch_size, 1))           # (bs, 1)

            

            for i in range(self.batch_size):
                img, text = self.next_sample()
                
                img = img.T
                img = np.expand_dims(img, -1)
                
                X_data[i] = img
                
                #if len(text_to_labels(text)) == 9:
                #    print(text)
                if '_' in text:
                    print(text)
                Y_data[i] = text_to_labels(text)
                label_length[i] = len(text)
                # X_data[i], Y_data[i] = self.process_data(X_data[i], Y_data[i])
            # dict 형태로 복사
            inputs = {
                'the_input': X_data,  # (bs, 128, 64, 1)
                'the_labels': Y_data,  # (bs, 8)
                'input_length': input_length, 
                'label_length': label_length  
            }
            outputs = {'ctc': np.zeros([self.batch_size])}   # (bs, 1) -> 모든 원소 0
            yield (inputs, outputs)

V. THỰC HIỆN TRAIN_FINE_TUNE

In [ ]:
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adadelta
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from Image_Generator import TextImageGenerator
from Model import get_Model
from parameter import *
K.set_learning_phase(0)
import os

In [ ]:
# Load pretrained model đối với synthetic data
model = get_Model(training=True)
try:
    model.load_weights('/home/anlab/License_plate/Recogniton_License_Area_Finetune_With_Real_Dataset_And_Augment/training_3/cp-0100.hdf5')
    
    print("...Previous weight data...")
except:
    print("...New weight data...")
    pass
# Do hôm qua em có soát lại thì theo source code của họ, mô hình chỉ đóng băng phần convolution và train với lớp Dense và LSTM, nên em có mở thêm 10 lớp conv để  mô hình học tiếp. Tổng cộng có 25 lớp conv, mở thêm 10 lớp, đóng lại 15 lớp
for layer in model.layers[:-15]:
    layer.trainable = False

In [ ]:
# Thư mục lưu checkpoint
checkpoint_path = "/home/anlab/License_plate/Recogniton_License_Area_Finetune_With_Real_Dataset_And_Augment/training_3/cp-{epoch:04d}.hdf5"
checkpoint_dir = os.path.dirname(checkpoint_path)
checkpoint_dir

In [ ]:
train_file_path = '/content/drive/MyDrive/Recognition CRNN/Recognition part. CRNN KERAS/dataset/Pipeline_train_and_test/train_output/'
tiger_train = TextImageGenerator(train_file_path, img_w, img_h, batch_size, downsample_factor)
tiger_train.build_data("train")

valid_file_path = '/content/drive/MyDrive/Recognition CRNN/Recognition part. CRNN KERAS/dataset/Pipeline_train_and_test/output_val/'
tiger_val = TextImageGenerator(valid_file_path, img_w, img_h, val_batch_size, downsample_factor)
tiger_val.build_data("val")

ada = Adadelta()
step = int(tiger_train.n / batch_size)
early_stop = EarlyStopping(monitor='loss', min_delta=0.001, patience=4, mode='min', verbose=1)
#checkpoint = ModelCheckpoint(filepath='Epochs/LSTM+BN5--{epoch:02d}--{val_loss:.3f}.hdf5', monitor='loss', verbose=1, mode='min', period=1)
checkpoint = ModelCheckpoint(filepath=checkpoint_path, monitor='loss', verbose=1, mode='min',save_freq=20*step, save_best_only=False, save_weights_only=False)
# model = get_Model(training=True)
# model.save_weights(checkpoint_path.format(epoch=0))
# the loss calc occurs elsewhere, so use a dummy lambda func for the loss
model.compile(loss={'ctc': lambda y_true, y_pred: y_pred}, optimizer=ada)

In [ ]:
# captures output of softmax so we can decode the output during visualization
history = model.fit_generator(generator=tiger_train.next_batch(),
                    steps_per_epoch=int(tiger_train.n / batch_size),
                    epochs=300,
                    callbacks=[checkpoint],
                    validation_data=tiger_val.next_batch_val(),
                    validation_steps=int(tiger_val.n / val_batch_size),
                    initial_epoch=200)

VI. TEST TRÊN TẬP OUTPUT_VAL

In [ ]:
import cv2
import itertools, os, time
import numpy as np
from Model import get_Model
from parameter import letters
import argparse
from tensorflow.keras import backend as K
K.set_learning_phase(0)

In [ ]:
def decode_label(out):
    # out : (1, 32, 42)
    out_best = list(np.argmax(out[0, 2:], axis=1))  # get max index -> len = 32
    out_best = [k for k, g in itertools.groupby(out_best)]  # remove overlap value
    #print(out_best)
    outstr = ''
    for i in out_best:
        if i < len(letters):
            #print("->" + letters[i])
            outstr += letters[i]
    return outstr

In [ ]:
model = get_Model(training=False)
model.load_weights(r'/home/anlab/License_plate/Recogniton_License_Area_Finetune_With_Real_Dataset_And_Augment/training_2/cp-0020.hdf5')
model.summary()

In [ ]:
def process_image(image):
    img = cv2.imread(image, cv2.IMREAD_GRAYSCALE)
    img_pred = img.astype(np.float32)
    img_pred = cv2.resize(img_pred, (128, 64))
    img_pred = (img_pred / 255.0) * 2.0 - 1.0
    img_pred = img_pred.T
    img_pred = np.expand_dims(img_pred, axis=-1)
    img_pred = np.expand_dims(img_pred, axis=0)
    return img_pred

In [ ]:
def edit_distance(s1, s2):
    m=len(s1)+1
    n=len(s2)+1

    tbl = {}
    for i in range(m): tbl[i,0]=i
    for j in range(n): tbl[0,j]=j
    for i in range(1, m):
        for j in range(1, n):
            cost = 0 if s1[i-1] == s2[j-1] else 1
            tbl[i,j] = min(tbl[i, j-1]+1, tbl[i-1, j]+1, tbl[i-1, j-1]+cost)

    return tbl[i,j]

In [ ]:
image_test_file = os.listdir('/content/drive/MyDrive/Recognition CRNN/Recognition part. CRNN KERAS/dataset/Pipeline_train_and_test/output_val/')
path_test_image = '/content/drive/MyDrive/Recognition CRNN/Recognition part. CRNN KERAS/dataset/Pipeline_train_and_test/output_val/'

In [ ]:
sum_cer = []
total = 0
acc = 0
letter_total = 0
letter_acc = 0
for image in image_test_file:
    # choice = random.randint(0,9999)
    path = path_test_image + image
    
    img_pred = process_image(path)
    net_out_value = model.predict(img_pred)
    pred_texts = decode_label(net_out_value)
    newstr = pred_texts.replace("Z", "")

    # label = image_test_file[i][0: len(image_test_file[i])-4]

    string = ''
    for j in range(3):
        if image[j]>='0' and image[j]<='9':
          continue
        else:
          string += image[j]
    # if string.endswith('p'): string = string.rstrip(string[-1])
    if string.endswith('.'): string = string.rstrip(string[-1])
    print('Predicted: '+newstr +" "+'Label: '+string)
    cer = edit_distance(newstr, string)
    error = cer/max(len(newstr), len(string))
    

    for i in range(min(len(newstr), len(string))):
    #ti = test_img[i][:-9]
        if newstr[i] == string[i]:
            letter_acc += 1
    letter_total += max(len(newstr), len(string))
    if newstr == string:
        acc += 1
    total += 1
    sum_cer.append(error)

In [ ]:
cer_rate = sum(sum_cer) / len(image_test_file)
print("CER: ",cer_rate)

In [ ]:
print("ACC : ", acc / total)
print("letter ACC : ", letter_acc / letter_total)